In [1]:
from edgar import *

In [2]:
set_identity("Zane Lim zyuanlim@gmail.com")

[17:47:14] INFO     Identity of the Edgar REST client set to [Zane Lim zyuanlim@gmail.com]              ]8;id=46156;file:///Users/zanelim/Projects/generativeai-worldcup/venv/lib/python3.11/site-packages/edgar/core.py\core.py]8;;\:]8;id=276214;file:///Users/zanelim/Projects/generativeai-worldcup/venv/lib/python3.11/site-packages/edgar/core.py#154\154]8;;\

In [3]:
filings = get_filings(form=["10-K", "10-Q"])

In [4]:
filings

╭──────────────────────────────────────────────────── Filings ────────────────────────────────────────────────────╮
│                                                                                                                 │
│        form     company                                   cik       filing_date   accession_number              │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────         │
│   0    10-K     AERWINS Technologies Inc.                 1855631   2024-04-30    0001493152-24-017204          │
│   1    10-K     Acreage Holdings, Inc.                    1762359   2024-04-30    0001762359-24-000023          │
│   2    10-K     Gaucho Group Holdings, Inc.               1559998   2024-04-30    0001493152-24-017110          │
│   3    10-K     MIND TECHNOLOGY, INC                      926423    2024-04-30    0001437749-24-013973          │
│   4    10-K     STREAMLINE HEALTH SOLUTIONS INC.          1008586   20

In [5]:
filing_metadata = []
for filing in filings:
    filing_info = filing.to_dict()
    metadata = {k: v for k, v in filing_info.items() if k in ['accession_no', 'form', 'filing_date']}
    filing_entity = filing.get_entity().to_dict()
    for k, v in filing_entity.items():
        if k in ['cik', 'name', 'display_name', 'tickers', 'exchanges', 'industry',' category']:
            metadata[k] = v
    filing_metadata.append(metadata)
    with open(f"../data/mds/{metadata['accession_no']}.md", "w") as f:
        f.write(filing.markdown())

In [6]:
import pandas as pd
import os

In [7]:
df_metadata = pd.DataFrame(filing_metadata)

In [8]:
df_metadata.shape

(1857, 9)

In [9]:
df_metadata.duplicated(subset='accession_no').sum()

37

In [10]:
df_metadata.drop_duplicates(subset='accession_no', inplace=True)

In [11]:
df_metadata.shape

(1820, 9)

In [35]:
df_metadata.to_parquet('../data/metadata.parquet', index=False)

In [15]:
from databricks.sdk import WorkspaceClient

In [18]:
os.environ["DATABRICKS_CONFIG_PROFILE"] = "DEFAULT"

In [19]:
w = WorkspaceClient()

[21:45:31] INFO     loading DEFAULT profile from ~/.databrickscfg: host, token                        ]8;id=660951;file:///Users/zanelim/Projects/generativeai-worldcup/venv/lib/python3.11/site-packages/databricks/sdk/config.py\config.py]8;;\:]8;id=753968;file:///Users/zanelim/Projects/generativeai-worldcup/venv/lib/python3.11/site-packages/databricks/sdk/config.py#417\417]8;;\

In [26]:
from pathlib import Path

In [27]:
data_folder = Path("../data/mds")

In [31]:
volume_path = "/Volumes/llm_hackathon/default/edgar"

In [34]:
for md_file in data_folder.iterdir():
    with open(md_file, "rb") as f:
        w.files.upload(f"{volume_path}/mds/{md_file.name}", f, overwrite=True)